In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
df_rec = pd.read_csv('recipes_10k.csv')
df_rec.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,site
0,1934604,Hot Spiked Cider,"[""8 large dried apple rings"", ""Small handful d...","[""Place the dried apples and currants in a lar...",www.cookstr.com/recipes/hot-spiked-cider,Recipes1M,"[""apple cider"", ""dark rum"", ""calvados"", ""nutme...",www.cookstr.com
1,1947433,Zucchini Spread,"[""2 teaspoons wine vinegar"", ""1 teaspoon sugar...","[""Put wine vinegar, sugar, and salt in a zip-t...",www.food.com/recipe/zucchini-spread-443492,Recipes1M,"[""wine vinegar"", ""sugar"", ""onion"", ""white pepp...",www.food.com
2,1976795,Fresh Pea Relish with Tuna Steak,"[""4 (6-ounce) tuna steaks"", ""Salt and pepper"",...","[""Preheat grill or broiler."", ""Season tuna wit...",www.foodnetwork.com/recipes/emeril-lagasse/fre...,Recipes1M,"[""peas"", ""fresh mixed baby greens"", ""extra-vir...",www.foodnetwork.com
3,400797,Hamburger Help Mate,"[""1 to 3 c. noodles"", ""1 can cream of chicken ...","[""Brown hamburger and onion."", ""Cook noodles.""...",www.cookbooks.com/Recipe-Details.aspx?id=853236,Gathered,"[""cream of chicken soup"", ""noodles"", ""garlic s...",www.cookbooks.com
4,1888762,Grape Jam,"[""1 1/2 bunches Grapes"", ""300 grams Granulated...","[""Wash the grapes one by one, cut in half, the...",cookpad.com/us/recipes/148776-grape-jam,Recipes1M,"[""grapes"", ""sugar"", ""lemon juice""]",cookpad.com


In [45]:
import re

pattern_with_parenthese = r"^(\d+)\s*([\d\s\/]+)?\s*\((\d+\s*(?:\d+\/\d+)?)\s*(oz\.?)\.\)\s*([^\r\n]+)$"
pattern_without_parenthese = r"^(\d+)\s*(\S+)?\s*([^\r\n]+)$"

count = 0
ingredient_count = 0

#go to whole dataset and look at exceptions of the regrex expression
def reportExceptions(matches, ingredient):
  global count
  if (matches):
    return
  else:
    count = count + 1
    print(ingredient)


def normailzeUnits(unit:str):
  unit = unit.replace('ounces', 'oz.')
  
  if (len(unit) > 0 and not unit[0].isdigit()):
    
    unit = "1 " + unit
  return unit

recipe_dic = {}

def extractUnits(row):
  ingredients = row['ingredients']
  recipe_title = row['title']
  ingredients = ingredients.replace('"', '')
  ingredients = ingredients.replace('[', '')
  ingredients = ingredients.replace(']', '')
  ingredients = ingredients.split(',')
  global ingredient_count
  global count
  
  for i in ingredients:
    ingredient_count = ingredient_count + 1
    n = i.lstrip()
    n = normailzeUnits(n)
    # print(i)
    matches1 = re.match(pattern_with_parenthese, n)
    matches = re.match(pattern_without_parenthese, n)

    if (not matches1 and not matches):
      continue
    elif (matches1):
      amount = matches1.group(1) if matches1.group(1) else 0
      unit = matches1.group(3) if matches1.group(3) else "NA"
      ingredient = matches1.group(4) if matches1.group(4) else "NA"
      if(recipe_title in recipe_dic):
        recipe_dic[recipe_title].append({"amount": amount, "unit":unit, "ingredient": ingredient})
      else:
        recipe_dic[recipe_title] = [{"amount": amount, "unit":unit, "ingredient": ingredient}]
    elif (matches):
      amount = matches.group(1) if matches.group(1) else 0
      unit = matches.group(2) if matches.group(2) else "NA"
      ingredient = matches.group(3) if matches.group(3) else "NA"
      if(recipe_title in recipe_dic):
        recipe_dic[recipe_title].append({"amount": amount, "unit":unit, "ingredient": ingredient})
      else:
        recipe_dic[recipe_title] = [{"amount": amount, "unit":unit, "ingredient": ingredient}]
  
      
      
# extractUnits(df_rec['ingredients'].iloc[3])
df_rec.apply(extractUnits, axis=1)
print("ingredient_count: ", ingredient_count)
print("count: ", count)

print(len(recipe_dic.items()))

ingredient_count:  105918
count:  0
9034


In [47]:
print(recipe_dic["Hot Spiked Cider"])

[{'amount': '8', 'unit': 'large', 'ingredient': 'dried apple rings'}, {'amount': '1', 'unit': 'Small', 'ingredient': 'handful dried currants (optional)'}, {'amount': '4', 'unit': 'oz.', 'ingredient': 'calvados or applejack (see Straight Up)'}, {'amount': '64', 'unit': 'oz.', 'ingredient': 'fresh apple cider'}, {'amount': '12', 'unit': 'whole', 'ingredient': 'cloves'}, {'amount': '6', 'unit': 'whole', 'ingredient': 'star anise'}, {'amount': '3', 'unit': 'cinnamon', 'ingredient': 'sticks'}, {'amount': '1', 'unit': '/2', 'ingredient': 'teaspoon freshly grated nutmeg'}, {'amount': '2', 'unit': 'tablespoons', 'ingredient': 'brown sugar for dredging the apple slices'}, {'amount': '24', 'unit': 'oz.', 'ingredient': 'dark rum'}, {'amount': '8', 'unit': 'extra-long', 'ingredient': 'cinnamon sticks'}]


In [4]:
# df_rec = pd.read_csv("archive/recipes_data.csv")
# df_rec = df_rec.sample(n=10000)
# df_rec.to_csv("recipes_10k.csv")